In [14]:
import pandas as pd
import re

# Setup

In [15]:
def list2txt(arr,filename):
    with open(f'{filename}','w') as f:
        f.write('\n'.join(arr))

In [16]:
def replace_nums(label):
    if pd.isna(label):
        return None
    regex = r"[\s]?[0-9]+. "
    return re.sub(regex,"/",label)

In [17]:
def clean(df, dataset):
    if dataset == "manual":
        # 1. Remove NaN rows
        df = df.dropna(axis=0,how="all")
    elif dataset == "topics":
        df.rename({0:"Website URL", 1:"Unformatted Labels"},axis=1,inplace=True)
        df = df.set_index("Website URL")
        df["Slash Labels"] = df["Unformatted Labels"].apply(replace_nums)
    return df

In [18]:
def extract_labels(label_str):
    '''
    Extract labels from a singular cell
    '''
    if pd.isna(label_str):
        return []
    return label_str.split("/")[1:] # Remove the first empty one

def combine_labels(df, label_count):
    '''
    Returns new column with the distinct labels after theyve been extracted
    '''
    
    # Combine labels in all the rows
    label_list = df[f"Label 1"].apply(extract_labels)
    for i in range(2,label_count+1):
        label_list += df[f"Label {i}"].apply(extract_labels)
        
    label_list = label_list.apply(lambda arr : list(set(arr))) # Remove duplicates
    return label_list

In [19]:
manual_df = pd.read_csv("manual_labelling.csv",skiprows=3,index_col="Website URL")
manual_df

,Label 1,Label 2,Label 3
Website URL,,,
trade.atlantiscex.com,/Finance/Financial Planning & Management,/Finance/Investing/Currencies & Foreign Exchange,/Business & Industrial/Advertising & Marketing...
w2.brreg.no,NaN,NaN,NaN
sadlierconnect.com,/Jobs & Education/Education,/Jobs & Education/Education/Colleges & Univers...,/Reference/Educational Resources
webcampus.fdu.edu,/Jobs & Education/Education,/Jobs & Education/Education/Colleges & Univers...,/Reference/Educational Resources
www.softbank.jp,/Finance/Investing/Currencies & Foreign Exchange,/Finance/Credit Cards,/Finance/Accounting & Auditing/Tax Preparation...
...,...,...,...
www.veepee.it,NaN,NaN,NaN
www.sephora.de,NaN,NaN,NaN
thothub.to,NaN,NaN,NaN


In [20]:
topics_df = pd.read_csv("topics_labelling.csv",header=None)
topics_df

,0,1
0,trade.atlantiscex.com,NaN
1,sadlierconnect.com,1. Arts & entertainment
2,webcampus.fdu.edu,229. Colleges & universities
3,www.softbank.jp,215. Internet & telecom 218. Phone service pro...
4,www.camif.fr,207. Home & garden
...,...,...
410,www.yannick.com.tw,1. Arts & entertainment 289. Shopping
411,www.langkahindopools.org,275. Reference
412,www.ngengesport.cd,243. News 299. Sports
413,doctruyen3q.site,100. Books & literature 1. Arts & entertainment


In [21]:
manual_df = clean(manual_df, "manual")
list2txt(list(manual_df.index), "urls.txt")

if "Labels" not in manual_df.columns:
    manual_df["Labels"] = combine_labels(manual_df, 3)
manual_df = manual_df[["Labels"]] # Dont need the other labels now

/home/rumi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:
manual_df

,Labels
Website URL,
trade.atlantiscex.com,"[Business & Industrial, Advertising & Marketin..."
sadlierconnect.com,"[Colleges & Universities, Education, Education..."
webcampus.fdu.edu,"[Colleges & Universities, Education, Education..."
www.softbank.jp,"[Credit Cards, Investing, Finance, Currencies ..."
www.camif.fr,"[Household Supplies, Home Improvement, Home & ..."
...,...
www.yannick.com.tw,"[Shopping, Food & Drink, Cooking & Recipes]"
www.langkahindopools.org,"[Online Communities, Finance, Sports]"
www.ngengesport.cd,"[Soccer, Finance, Sports]"


In [23]:
topics_df = clean(topics_df,"topics")
if "Labels" not in topics_df.columns:
    topics_df["Labels"] = topics_df["Slash Labels"].apply(extract_labels)
topics_df = topics_df[["Labels"]]
topics_df

,Labels
Website URL,
trade.atlantiscex.com,[]
sadlierconnect.com,[Arts & entertainment]
webcampus.fdu.edu,[Colleges & universities]
www.softbank.jp,"[Internet & telecom, Phone service providers]"
www.camif.fr,[Home & garden]
...,...
www.yannick.com.tw,"[Arts & entertainment, Shopping]"
www.langkahindopools.org,[Reference]
www.ngengesport.cd,"[News, Sports]"


# Comparison

In [31]:
combined_df = topics_df
combined_df.rename({"Labels":"Topics"},axis=1,inplace=True)
combined_df["Manual"] = manual_df["Labels"]
combined_df

,Topics,Manual
Website URL,,
trade.atlantiscex.com,[],"[Business & Industrial, Advertising & Marketin..."
sadlierconnect.com,[Arts & entertainment],"[Colleges & Universities, Education, Education..."
webcampus.fdu.edu,[Colleges & universities],"[Colleges & Universities, Education, Education..."
www.softbank.jp,"[Internet & telecom, Phone service providers]","[Credit Cards, Investing, Finance, Currencies ..."
www.camif.fr,[Home & garden],"[Household Supplies, Home Improvement, Home & ..."
...,...,...
www.yannick.com.tw,"[Arts & entertainment, Shopping]","[Shopping, Food & Drink, Cooking & Recipes]"
www.langkahindopools.org,[Reference],"[Online Communities, Finance, Sports]"
www.ngengesport.cd,"[News, Sports]","[Soccer, Finance, Sports]"
